# Extract Features

1. Read csv files and load as dfs
2. Combine dfs
3. Get POS, NER
4. Save in new features_df
5. Get semantic cosine similarity

In [1]:
import os, sys

import pandas as pd

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

import log_files
from log_files import LogData
from data_processing import DataProcessing
from feature_extraction import SpacyFeatureExtraction

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
log_file_path = "data/prediction_logs"
predictions = True
df = log_files.read_data(notebook_dir, log_file_path, predictions)
df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/prediction_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/prediction_logs/batch_1-prediction
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/prediction_logs/batch_1-prediction/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/prediction_logs/batch_2-prediction
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/prediction_logs/batch_2-prediction/batch_2-from_df.csv


,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,1
1,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,2
2,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,3
3,"According to Goldman Sachs, the research and development expenses at Facebook would fall in 2025.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,4
4,"In 21 August 2024, Morgan Stanley envisions that the gross profit at Johnson & Johnson has some probability to remain stable.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,5
5,"The stock price at Visa should stay same in Q2 of 2026, according to Wells Fargo.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,6
6,JPMorgan forecasts that the revenue at Microsoft potentially decrease in Q3 of 2027.,1,finance,llama-3.3-70b-instruct,NAVI_GATOR,0,1


In [4]:
# df∫

In [5]:
df[2 : 3]

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
2,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,3


- issue: `obesity rate` is not being captures by POS or NER.

In [6]:
only_predictions = DataProcessing.df_to_list(df, 'Base Sentence')
print(only_predictions)

sfe_class = SpacyFeatureExtraction(df, "Base Sentence")
word_leveL_disable_components = ["lemmatizer"]
word_level_pos_tags, word_level_pos_mappings, word_level_ner_entities, word_level_ner_mappings =  sfe_class.extract_features(data=only_predictions, disable_components=word_leveL_disable_components)
all_word_level_pos_df = DataProcessing.convert_to_df(word_level_pos_tags, mapping=word_level_pos_mappings)
all_word_level_ner_df = DataProcessing.convert_to_df(word_level_ner_entities, word_level_ner_mappings)
pred_pos_ner_features = [all_word_level_pos_df, all_word_level_ner_df]
pred_pos_ner_features_df = DataProcessing.concat_dfs(pred_pos_ner_features, axis=1, ignore_index=False)
pred_pos_ner_features_df.head()

['JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.', 'On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.', 'Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.', 'According to Goldman Sachs, the research and development expenses at Facebook would fall in 2025.', 'In 21 August 2024, Morgan Stanley envisions that the gross profit at Johnson & Johnson has some probability to remain stable.', 'The stock price at Visa should stay same in Q2 of 2026, according to Wells Fargo.', 'JPMorgan forecasts that the revenue at Microsoft potentially decrease in Q3 of 2027.', 'On August 25, 2024, to September 25, 2025, Citigroup speculates the net profit at Johnson & Johnson will likely increase.', 'Bank of America predicts on 2024-08-21, the operating income at Visa may rise.', 'According to Goldman Sachs, the research and development expenses at Alphabet would fall in 2029 Q2.', 'In 21 August 

,ADJ_3,ADV_1,NOUN_10,PRON_1,SYM_2,DET_2,ADJ_4,NOUN_3,CCONJ_2,DET_5,NOUN_2,AUX_1,PUNCT_1,ADP_5,VERB_2,DET_1,ADJ_2,PUNCT_4,PROPN_1,NUM_4,DET_4,PUNCT_5,DET_3,VERB_4,PUNCT_2,NUM_2,ADP_2,ADP_4,SCONJ_1,PROPN_2,SYM_1,PROPN_3,PROPN_5,ADP_6,ADP_1,PRON_2,PROPN_6,PUNCT_3,CCONJ_1,NOUN_5,NOUN_8,NOUN_11,NUM_1,VERB_3,NUM_3,ADJ_1,PROPN_7,ADP_3,PART_1,VERB_1,NOUN_6,NOUN_9,NOUN_1,NOUN_7,NOUN_4,PROPN_4,ORG_2,MONEY_1,CARDINAL_1,NORP_1,ORG_3,GPE_1,ORG_1,DATE_1,GPE_2,PERSON_2,FAC_1,PERSON_1,DATE_2,PRODUCT_1,LOC_1,EVENT_1,TIME_1
0,NaN,potentially,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,decrease,NaN,.,NaN,NaN,the,NaN,NaN,JPMorgan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,in,NaN,that,Chase,NaN,Amazon,NaN,NaN,at,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2027,NaN,NaN,net,NaN,of,NaN,forecasts,NaN,NaN,profit,NaN,NaN,Q3,Amazon,NaN,NaN,NaN,NaN,Q3,JPMorgan Chase,2027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,likely,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,will,",",NaN,increase,the,NaN,NaN,August,NaN,NaN,NaN,NaN,NaN,",",2024,of,NaN,NaN,Bank,NaN,America,NaN,NaN,On,NaN,NaN,.,NaN,NaN,NaN,NaN,21,NaN,NaN,NaN,NaN,at,NaN,speculates,NaN,NaN,revenue,NaN,NaN,Microsoft,Microsoft,NaN,NaN,NaN,NaN,NaN,Bank of America,"August 21, 2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,income,may,",",NaN,rise,the,NaN,NaN,Citigroup,NaN,NaN,NaN,NaN,NaN,.,08,at,NaN,NaN,Alphabet,-,NaN,NaN,NaN,on,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024,NaN,21,NaN,NaN,NaN,NaN,predicts,NaN,NaN,operating,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Citigroup,2024-08-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,expenses,NaN,NaN,development,would,",",NaN,fall,the,NaN,NaN,Goldman,NaN,NaN,NaN,NaN,NaN,.,NaN,at,NaN,NaN,Sachs,NaN,Facebook,NaN,NaN,to,NaN,NaN,NaN,and,NaN,NaN,NaN,2025,NaN,NaN,NaN,NaN,in,NaN,According,NaN,NaN,research,NaN,NaN,NaN,Facebook,NaN,NaN,NaN,NaN,NaN,Goldman Sachs,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,some,NaN,NaN,NaN,NaN,probability,NaN,",",NaN,has,the,stable,NaN,August,NaN,NaN,NaN,NaN,NaN,.,2024,at,NaN,that,Morgan,NaN,Stanley,Johnson,NaN,In,NaN,NaN,NaN,&,NaN,NaN,NaN,21,remain,NaN,gross,NaN,NaN,to,envisions,NaN,NaN,profit,NaN,NaN,Johnson,Johnson & Johnson,NaN,NaN,NaN,NaN,NaN,Morgan Stanley,21 August 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Observations

In [7]:
log_file_path = "data/observation_logs"
predictions = False
observations_df = log_files.read_data(notebook_dir, log_file_path, predictions)
observations_df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/observation_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/observation_logs/batch_1-observation
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/observation_logs/batch_1-observation/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/observation_logs/batch_2-observation
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/misc_experiments/../data/observation_logs/batch_2-observation/batch_2-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/m

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.,0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,1
1,"On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,2
2,"A young investor predicts on 2025-03-15, the S&P 500 index may rise.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,3
3,"According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,4
4,"In 2027-01-01 to 2027-12-31, Wells Fargo envisions that the interest rates at the Federal Reserve have some probability to remain stable.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,5
5,"The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,6
6,JPMorgan observed that the net profit at Microsoft had risen in September 2023.,0,finance,llama-3.3-70b-instruct,NAVI_GATOR,0,1


In [8]:
only_observations = DataProcessing.df_to_list(observations_df, 'Base Sentence')
# print(only_observations)
word_level_pos_tags, word_level_pos_mappings, word_level_ner_entities, word_level_ner_mappings = sfe_class.extract_features(data=only_observations, disable_components=word_leveL_disable_components)
all_word_level_pos_df = DataProcessing.convert_to_df(word_level_pos_tags, mapping=word_level_pos_mappings)
all_word_level_ner_df = DataProcessing.convert_to_df(word_level_ner_entities, word_level_ner_mappings)
obs_pos_ner_features = [all_word_level_pos_df, all_word_level_ner_df]
obs_pos_ner_features_df = DataProcessing.concat_dfs(obs_pos_ner_features, axis=1, ignore_index=False)
obs_pos_ner_features_df.head(3)

Spacy Doc (0):  The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.
Spacy Doc (1):  On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.
Spacy Doc (2):  A young investor predicts on 2025-03-15, the S&P 500 index may rise.
Spacy Doc (3):  According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.


,ADJ_3,PROPN_8,PRON_1,SYM_2,DET_6,DET_2,ADJ_4,NOUN_3,AUX_2,SYM_3,PROPN_10,DET_5,NOUN_2,AUX_1,SCONJ_2,AUX_3,PUNCT_1,PUNCT_6,ADP_5,VERB_2,NUM_5,DET_1,ADJ_2,PUNCT_4,PROPN_1,NUM_4,PROPN_9,DET_4,PUNCT_5,DET_3,VERB_4,PUNCT_2,NUM_2,ADP_2,ADP_7,ADP_4,PROPN_4,SCONJ_1,ADP_9,PROPN_2,SYM_1,PROPN_3,ADJ_5,PROPN_5,ADP_6,ADP_1,PROPN_6,PUNCT_3,NUM_6,CCONJ_1,SYM_4,NOUN_5,NOUN_8,ADV_2,NUM_1,VERB_3,NUM_3,ADJ_1,PROPN_7,ADP_3,PART_2,PART_1,ADP_8,VERB_1,NOUN_6,NOUN_9,NOUN_1,NOUN_7,NOUN_4,ADV_1,ORG_2,ORG_1,DATE_1,ORDINAL_1,CARDINAL_2,TIME_1,CARDINAL_1,ORG_4,LOC_1,NORP_1,GPE_1,DATE_2,LAW_1,FAC_1,EVENT_1,ORG_3,GPE_2,PERSON_1,PERCENT_1,QUANTITY_1,DATE_3
0,NaN,NaN,NaN,NaN,NaN,the,NaN,income,NaN,NaN,NaN,NaN,operating,had,NaN,NaN,.,NaN,NaN,increased,NaN,The,first,NaN,Goldman,NaN,NaN,NaN,NaN,the,NaN,NaN,NaN,at,NaN,of,NaN,that,NaN,Sachs,NaN,Tesla,NaN,NaN,NaN,at,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024,NaN,NaN,financial,NaN,in,NaN,NaN,NaN,observed,NaN,NaN,analyst,NaN,quarter,NaN,Tesla,Goldman Sachs,the first quarter of 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,price,will,NaN,NaN,",",NaN,NaN,rise,08,the,NaN,NaN,Morgan,2025,NaN,NaN,NaN,NaN,NaN,.,08,at,NaN,NaN,NaN,NaN,NaN,Stanley,-,Amazon,NaN,NaN,NaN,On,NaN,NaN,20,NaN,-,NaN,NaN,NaN,2024,NaN,20,NaN,NaN,NaN,NaN,to,NaN,speculates,NaN,NaN,stock,NaN,NaN,likely,Amazon,Morgan Stanley,2024-08-20 to,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,-,NaN,the,NaN,NaN,NaN,NaN,NaN,NaN,index,may,NaN,NaN,",",NaN,NaN,rise,NaN,A,NaN,NaN,S&P,500,NaN,NaN,NaN,NaN,NaN,.,03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN,on,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,NaN,15,young,NaN,NaN,NaN,NaN,NaN,predicts,NaN,NaN,investor,NaN,NaN,NaN,NaN,NaN,2025-03-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Both

- Create a knowledge graph
    - Nodes: words
    - Edges: connection to other words (same/diff sentence)
- Look at code from Graphbreeding project on 2019 Mac

In [9]:
pred_sentence_features_df = DataProcessing.include_sentence_and_label(pred_pos_ner_features_df, df)
pred_sentence_features_df.head(3)
pred_profit_filt = (pred_pos_ner_features_df.NOUN_1.values == 'profit')
pred_profit_df = pred_pos_ner_features_df.loc[pred_profit_filt == True]
pred_profit_df

,Base Sentence,Sentence Label,ADJ_3,ADV_1,NOUN_10,PRON_1,SYM_2,DET_2,ADJ_4,NOUN_3,CCONJ_2,DET_5,NOUN_2,AUX_1,PUNCT_1,ADP_5,VERB_2,DET_1,ADJ_2,PUNCT_4,PROPN_1,NUM_4,DET_4,PUNCT_5,DET_3,VERB_4,PUNCT_2,NUM_2,ADP_2,ADP_4,SCONJ_1,PROPN_2,SYM_1,PROPN_3,PROPN_5,ADP_6,ADP_1,PRON_2,PROPN_6,PUNCT_3,CCONJ_1,NOUN_5,NOUN_8,NOUN_11,NUM_1,VERB_3,NUM_3,ADJ_1,PROPN_7,ADP_3,PART_1,VERB_1,NOUN_6,NOUN_9,NOUN_1,NOUN_7,NOUN_4,PROPN_4,ORG_2,MONEY_1,CARDINAL_1,NORP_1,ORG_3,GPE_1,ORG_1,DATE_1,GPE_2,PERSON_2,FAC_1,PERSON_1,DATE_2,PRODUCT_1,LOC_1,EVENT_1,TIME_1
0,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,1,NaN,potentially,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,decrease,NaN,.,NaN,NaN,the,NaN,NaN,JPMorgan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,in,NaN,that,Chase,NaN,Amazon,NaN,NaN,at,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2027,NaN,NaN,net,NaN,of,NaN,forecasts,NaN,NaN,profit,NaN,NaN,Q3,Amazon,NaN,NaN,NaN,NaN,Q3,JPMorgan Chase,2027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"In 21 August 2024, Morgan Stanley envisions that the gross profit at Johnson & Johnson has some probability to remain stable.",1,NaN,NaN,NaN,NaN,NaN,some,NaN,NaN,NaN,NaN,probability,NaN,",",NaN,has,the,stable,NaN,August,NaN,NaN,NaN,NaN,NaN,.,2024,at,NaN,that,Morgan,NaN,Stanley,Johnson,NaN,In,NaN,NaN,NaN,&,NaN,NaN,NaN,21,remain,NaN,gross,NaN,NaN,to,envisions,NaN,NaN,profit,NaN,NaN,Johnson,Johnson & Johnson,NaN,NaN,NaN,NaN,NaN,Morgan Stanley,21 August 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"On August 25, 2024, to September 25, 2025, Citigroup speculates the net profit at Johnson & Johnson will likely increase.",1,NaN,likely,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,will,",",NaN,increase,the,NaN,",",August,2025,NaN,.,NaN,NaN,",",2024,to,NaN,NaN,September,NaN,Citigroup,Johnson,NaN,On,NaN,NaN,",",&,NaN,NaN,NaN,25,NaN,25,net,NaN,at,NaN,speculates,NaN,NaN,profit,NaN,NaN,Johnson,Johnson & Johnson,NaN,NaN,NaN,NaN,NaN,Citigroup,"August 25, 2024",NaN,NaN,NaN,NaN,"September 25, 2025",NaN,NaN,NaN,NaN
10,"In 21 August 2024, Wells Fargo envisions that the gross profit at Procter & Gamble has some probability to remain stable.",1,NaN,NaN,NaN,NaN,NaN,some,NaN,NaN,NaN,NaN,probability,NaN,",",NaN,has,the,stable,NaN,August,NaN,NaN,NaN,NaN,NaN,.,2024,at,NaN,that,Wells,NaN,Fargo,Gamble,NaN,In,NaN,NaN,NaN,&,NaN,NaN,NaN,21,remain,NaN,gross,NaN,NaN,to,envisions,NaN,NaN,profit,NaN,NaN,Procter,Procter & Gamble,NaN,NaN,NaN,NaN,NaN,Wells Fargo,21 August 2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,"On August 21, 2024, Goldman Sachs speculates that the net profit at Microsoft will likely increase in the following fiscal year.",1,NaN,likely,NaN,NaN,NaN,the,NaN,NaN,NaN,NaN,year,will,",",NaN,increase,the,fiscal,NaN,August,NaN,NaN,NaN,NaN,NaN,",",2024,at,NaN,that,Goldman,NaN,Sachs,NaN,NaN,On,NaN,NaN,.,NaN,NaN,NaN,NaN,21,following,NaN,net,NaN,in,NaN,speculates,NaN,NaN,profit,NaN,NaN,Microsoft,Microsoft,NaN,NaN,NaN,NaN,NaN,Goldman Sachs,"August 21, 2024",NaN,NaN,NaN,NaN,the following fiscal year,NaN,NaN,NaN,NaN
83,"The net profit at Amazon should stay same in 2024/08/21, according to a financial reporter.",1,financial,NaN,NaN,NaN,NaN,a,NaN,NaN,NaN,NaN,reporter,should,",",NaN,according,The,same,NaN,Amazon,NaN,NaN,NaN,NaN,NaN,.,NaN,in,NaN,NaN,NaN,NaN,NaN,NaN,NaN,at,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/08/21,NaN,NaN,net,NaN,to,NaN,stay,NaN,NaN,profit,NaN,NaN,NaN,NaN,NaN,2024/08/21,NaN,NaN,NaN,Amazon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
pred_profit_sentences = DataProcessing.df_to_list(pred_profit_df, 'Base Sentence')
# pred_profit_sentences_2 = DataProcessing.df_to_list(pred_profit_df, 'Base Sentence')
# pred_profit_sentences_3 = DataProcessing.df_to_list(pred_profit_df, 'Base Sentence')
# pred_profit_sentences_4 = DataProcessing.df_to_list(pred_profit_df, 'Base Sentence')


In [11]:
obs_profit_sentence_df = DataProcessing.include_sentence_and_label(obs_pos_ner_features_df, observations_df)
obs_profit_filt = (obs_pos_ner_features_df.NOUN_1.values == 'profit')
obs_profit_df = obs_pos_ner_features_df.loc[obs_profit_filt == True]
obs_profit_df

,Base Sentence,Sentence Label,ADJ_3,PROPN_8,PRON_1,SYM_2,DET_6,DET_2,ADJ_4,NOUN_3,AUX_2,SYM_3,PROPN_10,DET_5,NOUN_2,AUX_1,SCONJ_2,AUX_3,PUNCT_1,PUNCT_6,ADP_5,VERB_2,NUM_5,DET_1,ADJ_2,PUNCT_4,PROPN_1,NUM_4,PROPN_9,DET_4,PUNCT_5,DET_3,VERB_4,PUNCT_2,NUM_2,ADP_2,ADP_7,ADP_4,PROPN_4,SCONJ_1,ADP_9,PROPN_2,SYM_1,PROPN_3,ADJ_5,PROPN_5,ADP_6,ADP_1,PROPN_6,PUNCT_3,NUM_6,CCONJ_1,SYM_4,NOUN_5,NOUN_8,ADV_2,NUM_1,VERB_3,NUM_3,ADJ_1,PROPN_7,ADP_3,PART_2,PART_1,ADP_8,VERB_1,NOUN_6,NOUN_9,NOUN_1,NOUN_7,NOUN_4,ADV_1,ORG_2,ORG_1,DATE_1,ORDINAL_1,CARDINAL_2,TIME_1,CARDINAL_1,ORG_4,LOC_1,NORP_1,GPE_1,DATE_2,LAW_1,FAC_1,EVENT_1,ORG_3,GPE_2,PERSON_1,PERCENT_1,QUANTITY_1,DATE_3
3,"According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.",0,NaN,NaN,NaN,NaN,NaN,the,NaN,NaN,NaN,NaN,NaN,NaN,quarter,would,NaN,NaN,",",NaN,of,fall,NaN,the,second,NaN,Bank,NaN,NaN,NaN,NaN,NaN,NaN,.,NaN,of,NaN,in,NaN,NaN,NaN,America,NaN,Microsoft,NaN,NaN,NaN,to,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2026,NaN,NaN,net,NaN,at,NaN,NaN,NaN,According,NaN,NaN,profit,NaN,NaN,NaN,Microsoft,Bank of America,the second quarter of 2026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,JPMorgan observed that the net profit at Microsoft had risen in September 2023.,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,had,NaN,NaN,.,NaN,NaN,risen,NaN,the,NaN,NaN,JPMorgan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,in,NaN,NaN,NaN,that,NaN,Microsoft,NaN,September,NaN,NaN,NaN,at,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,NaN,NaN,net,NaN,NaN,NaN,NaN,NaN,observed,NaN,NaN,profit,NaN,NaN,NaN,Microsoft,JPMorgan,September 2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"According to Google, the gross profit at Facebook would fall in the second quarter of 2026.",0,NaN,NaN,NaN,NaN,NaN,the,NaN,NaN,NaN,NaN,NaN,NaN,quarter,would,NaN,NaN,",",NaN,NaN,fall,NaN,the,second,NaN,Google,NaN,NaN,NaN,NaN,NaN,NaN,.,NaN,at,NaN,of,NaN,NaN,NaN,Facebook,NaN,NaN,NaN,NaN,NaN,to,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2026,NaN,NaN,gross,NaN,in,NaN,NaN,NaN,According,NaN,NaN,profit,NaN,NaN,NaN,Facebook,Google,the second quarter of 2026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,"According to Microsoft, the net profit at Facebook would fall in the second quarter of 2026.",0,NaN,NaN,NaN,NaN,NaN,the,NaN,NaN,NaN,NaN,NaN,NaN,quarter,would,NaN,NaN,",",NaN,NaN,fall,NaN,the,second,NaN,Microsoft,NaN,NaN,NaN,NaN,NaN,NaN,.,NaN,at,NaN,of,NaN,NaN,NaN,Facebook,NaN,NaN,NaN,NaN,NaN,to,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2026,NaN,NaN,net,NaN,in,NaN,NaN,NaN,According,NaN,NaN,profit,NaN,NaN,NaN,Facebook,Microsoft,the second quarter of 2026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
232,"On 11/15/2022, the Wall Street Journal speculated the gross profit at Microsoft would likely increase.",0,NaN,NaN,NaN,NaN,NaN,the,NaN,NaN,NaN,NaN,NaN,NaN,NaN,would,NaN,NaN,",",NaN,NaN,increase,NaN,the,NaN,NaN,Wall,NaN,NaN,NaN,NaN,NaN,NaN,.,NaN,at,NaN,NaN,Microsoft,NaN,NaN,Street,NaN,Journal,NaN,NaN,NaN,On,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11/15/2022,NaN,NaN,gross,NaN,NaN,NaN,NaN,NaN,speculated,NaN,NaN,profit,NaN,NaN,likely,Microsoft,the Wall Street Journal,11/15/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234,"According to JPMorgan Chase, the net profit at Tesla would fall in 06/2022.",0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,would,NaN,NaN,",",NaN,NaN,fall,NaN,the,NaN,NaN,JPMorgan,NaN,NaN,NaN,NaN,NaN,NaN,.,NaN,at,NaN,NaN,NaN,NaN,NaN,Chase,NaN,Tesla,NaN,NaN,NaN,to,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/2022,NaN,NaN,net,NaN,in,NaN,NaN,NaN,According,NaN,NaN,profit,NaN,NaN,NaN,Tesla,JPMorgan Chase,NaN,NaN,NaN,NaN,06/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
413,"Citigroup noted that the net profit at Microsoft had risen significantly from September 10, 2023, to September 10, 2024.",0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,had,NaN,NaN,",",NaN,NaN,risen,NaN,the,NaN,.,Citigrou

In [12]:
obs_profit_sentences = DataProcessing.df_to_list(obs_profit_df, 'Base Sentence')
obs_profit_sentences

['According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.',
 'JPMorgan observed that the net profit at Microsoft had risen in September 2023.',
 'According to Google, the gross profit at Facebook would fall in the second quarter of 2026.',
 'According to Microsoft, the net profit at Facebook would fall in the second quarter of 2026.',
 'On 11/15/2022, the Wall Street Journal speculated the gross profit at Microsoft would likely increase.',
 'According to JPMorgan Chase, the net profit at Tesla would fall in 06/2022.',
 'Citigroup noted that the net profit at Microsoft had risen significantly from September 10, 2023, to September 10, 2024.',
 'JPMorgan Chase noted that the net profit at Amazon had risen significantly from September 10, 2023, to September 10, 2024.',
 "Google's net profit should stay the same in Q2 2027, according to financial records.",
 'JPMorgan Chase observed that the net profit at Amazon had remained stable in Q2 2026.',
 

In [13]:
pred_obs_dict = {}
for pred_sent_idx in range(len(pred_profit_sentences)):
    pred_sentence = pred_profit_sentences[pred_sent_idx]
    pred_obs_dict[pred_sentence] = obs_profit_sentences
pred_obs_dict

{'JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.': ['According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.',
  'JPMorgan observed that the net profit at Microsoft had risen in September 2023.',
  'According to Google, the gross profit at Facebook would fall in the second quarter of 2026.',
  'According to Microsoft, the net profit at Facebook would fall in the second quarter of 2026.',
  'On 11/15/2022, the Wall Street Journal speculated the gross profit at Microsoft would likely increase.',
  'According to JPMorgan Chase, the net profit at Tesla would fall in 06/2022.',
  'Citigroup noted that the net profit at Microsoft had risen significantly from September 10, 2023, to September 10, 2024.',
  'JPMorgan Chase noted that the net profit at Amazon had risen significantly from September 10, 2023, to September 10, 2024.',
  "Google's net profit should stay the same in Q2 2027, according to financial

In [14]:
import spacy

load_nlp_model = spacy.load("en_core_web_sm")

all_mappings = {} # Mapping of predictions to observations
pom_mappings = {} # Mapping of predictions to observations to metrics/scoresl pos can be misleading

oms = []
for prediction in pred_obs_dict.keys():
    print(f"Prediction: {prediction}")

    om_mappings = {} # Mapping of observations to metrics/scores

    for observation in pred_obs_dict[prediction]:
        print(f"    Observation: {observation}")

        # Calculate the similarity score
        prediction_doc = load_nlp_model(prediction)
        observation_doc = load_nlp_model(observation)
        similarity = prediction_doc.similarity(observation_doc)
        print(f"    Similarity: {similarity}")
        om_mappings[observation] = [similarity]
        print()   
        # print(f"    OM Mapping: {om_mappings}")
    pom_mappings[prediction] = om_mappings

# Flatten the dictionary into a list of rows
data = []
for prediction, observations in pom_mappings.items():
    for observation, scores in observations.items():
        data.append([prediction, observation, scores[0]])

# Create the DataFrame
df = pd.DataFrame(data, columns=['Prediction', 'Observation', 'Spacy Similarity'])

# Display the DataFrame
df['Scores'] = "Similarity"
df

Prediction: JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.
    Observation: According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.
    Similarity: 0.706486701965332

    Observation: JPMorgan observed that the net profit at Microsoft had risen in September 2023.
    Similarity: 0.7725691795349121

    Observation: According to Google, the gross profit at Facebook would fall in the second quarter of 2026.
    Similarity: 0.6635857224464417

    Observation: According to Microsoft, the net profit at Facebook would fall in the second quarter of 2026.
    Similarity: 0.6858891844749451

    Observation: On 11/15/2022, the Wall Street Journal speculated the gross profit at Microsoft would likely increase.
    Similarity: 0.6503056287765503

    Observation: According to JPMorgan Chase, the net profit at Tesla would fall in 06/2022.
    Similarity: 0.6568577289581299

    Observation: Citigroup noted tha

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_80358/1625276997.py:20: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc)


    Similarity: 0.8545955419540405

    Observation: Google's net profit should stay the same in Q2 2027, according to financial records.
    Similarity: 0.5586764812469482

    Observation: JPMorgan Chase observed that the net profit at Amazon had remained stable in Q2 2026.
    Similarity: 0.5882915258407593

    Observation: Bank of America observed that on October 15, 2024, the net profit at General Motors fell sharply.
    Similarity: 0.8109949231147766

Prediction: In 21 August 2024, Wells Fargo envisions that the gross profit at Procter & Gamble has some probability to remain stable.
    Observation: According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.
    Similarity: 0.5780841112136841

    Observation: JPMorgan observed that the net profit at Microsoft had risen in September 2023.
    Similarity: 0.6776936054229736

    Observation: According to Google, the gross profit at Facebook would fall in the second quarter of 2026.
    Sim

,Prediction,Observation,Spacy Similarity,Scores
0,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.",0.706487,Similarity
1,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,JPMorgan observed that the net profit at Microsoft had risen in September 2023.,0.772569,Similarity
2,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"According to Google, the gross profit at Facebook would fall in the second quarter of 2026.",0.663586,Similarity
3,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"According to Microsoft, the net profit at Facebook would fall in the second quarter of 2026.",0.685889,Similarity
4,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"On 11/15/2022, the Wall Street Journal speculated the gross profit at Microsoft would likely increase.",0.650306,Similarity
5,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"According to JPMorgan Chase, the net profit at Tesla would fall in 06/2022.",0.656858,Similarity
6,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"Citigroup noted that the net profit at Microsoft had risen significantly from September 10, 2023, to September 10, 2024.",0.586532,Similarity
7,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"JPMorgan Chase noted that the net profit at Amazon had risen significantly from September 10, 2023, to September 10, 2024.",0.606574,Similarity
8,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"Google's net profit should stay the same in Q2 2027, according to financial records.",0.513798,Similarity
9,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,JPMorgan Chase observed that the net profit at Amazon had remained stable in Q2 2026.,0.779024,Similarity


In [15]:
data

[['JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.',
  'According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.',
  0.706486701965332],
 ['JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.',
  'JPMorgan observed that the net profit at Microsoft had risen in September 2023.',
  0.7725691795349121],
 ['JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.',
  'According to Google, the gross profit at Facebook would fall in the second quarter of 2026.',
  0.6635857224464417],
 ['JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.',
  'According to Microsoft, the net profit at Facebook would fall in the second quarter of 2026.',
  0.6858891844749451],
 ['JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.',
  'On 11/15/2022, the Wall Street Journal sp

In [16]:
table = pd.pivot_table(df, values='Spacy Similarity', index=['Prediction', 'Observation'],
                       columns=['Scores'])

- Only grouping by `p_a = profit`. 
- What if also group by sentiment?
    - fall/decrease ---  drastically different --- neutral bc dd doesn't indicate slope/how
    - fall --- plummeted, so similar

- What set of observations certify prediction x?
1. Collect data
    1. Real data
        1. Classify as prediction or observation
    2. Generate data
        1. Verify prediction or observation
2. Certify 
    1. Match observations with predictions
        1. Partial
        2. Full
    2. State which observations verify prediction as true or false

In [17]:
table

Scores                                                                                                                                                                                                                                                       Similarity
Prediction                                                                                                                       Observation                                                                                                                           
In 21 August 2024, Morgan Stanley envisions that the gross profit at Johnson & Johnson has some probability to remain stable.    According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.                           0.598909
                                                                                                                                 According to Google, the gross profit at Facebook would fall in the second quarter of 2026.                                   0.602676
                                                                                                                                 According to JPMorgan Chase, the net profit at Tesla would fall in 06/2022.                                                   0.600226
                                                                                                                                 According to Microsoft, the net profit at Facebook would fall in the second quarter of 2026.                                  0.598431
                                                                                                                                 Bank of America observed that on October 15, 2024, the net profit at General Motors fell sharply.                             0.656063
                                                                                                                                 Citigroup noted that the net profit at Microsoft had risen significantly from September 10, 2023, to September 10, 2024.      0.573934
                                                                                                                                 Google's net profit should stay the same in Q2 2027, according to financial records.                                          0.551721
                                                                                                                                 JPMorgan Chase noted that the net profit at Amazon had risen significantly from September 10, 2023, to September 10, 2024.    0.598704
                                                                                                                                 JPMorgan Chase observed that the net profit at Amazon had remained stable in Q2 2026.                                         0.759091
                                                                                                                                 JPMorgan observed that the net profit at Microsoft had risen in September 2023.                                               0.718996
                                                                                                                                 On 11/15/2022, the Wall Street Journal speculated the gross profit at Microsoft would likely increase.                        0.659232
In 21 August 2024, Wells Fargo envisions that the gross profit at Procter & Gamble has some probability to remain stable.        According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.                           0.578084
                                                                                                                                 According to Google, the gross profit at Facebook would fall in the second quarter of 2026.                                   0.574753
                                        

In [18]:
# Methods to alter Precision and Recall for prob rather than binary
    
#     https://en.wikipedia.org/wiki/Fuzzy_classification

# Support or deny classes

In [19]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# The sentences to encode
sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
]

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 384]

# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)
# tensor([[1.0000, 0.6660, 0.1046],
#         [0.6660, 1.0000, 0.1411],
#         [0.1046, 0.1411, 1.0000]])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(3, 384)
tensor([[1.0000, 0.6660, 0.1046],
        [0.6660, 1.0000, 0.1411],
        [0.1046, 0.1411, 1.0000]])


In [20]:
pred_profit_sentences

['JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.',
 'In 21 August 2024, Morgan Stanley envisions that the gross profit at Johnson & Johnson has some probability to remain stable.',
 'On August 25, 2024, to September 25, 2025, Citigroup speculates the net profit at Johnson & Johnson will likely increase.',
 'In 21 August 2024, Wells Fargo envisions that the gross profit at Procter & Gamble has some probability to remain stable.',
 'On August 21, 2024, Goldman Sachs speculates that the net profit at Microsoft will likely increase in the following fiscal year.',
 'The net profit at Amazon should stay same in 2024/08/21, according to a financial reporter.']

In [21]:
obs_profit_sentences

['According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.',
 'JPMorgan observed that the net profit at Microsoft had risen in September 2023.',
 'According to Google, the gross profit at Facebook would fall in the second quarter of 2026.',
 'According to Microsoft, the net profit at Facebook would fall in the second quarter of 2026.',
 'On 11/15/2022, the Wall Street Journal speculated the gross profit at Microsoft would likely increase.',
 'According to JPMorgan Chase, the net profit at Tesla would fall in 06/2022.',
 'Citigroup noted that the net profit at Microsoft had risen significantly from September 10, 2023, to September 10, 2024.',
 'JPMorgan Chase noted that the net profit at Amazon had risen significantly from September 10, 2023, to September 10, 2024.',
 "Google's net profit should stay the same in Q2 2027, according to financial records.",
 'JPMorgan Chase observed that the net profit at Amazon had remained stable in Q2 2026.',
 

In [22]:
my_sentences = pred_profit_sentences + obs_profit_sentences
my_sentences

['JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.',
 'In 21 August 2024, Morgan Stanley envisions that the gross profit at Johnson & Johnson has some probability to remain stable.',
 'On August 25, 2024, to September 25, 2025, Citigroup speculates the net profit at Johnson & Johnson will likely increase.',
 'In 21 August 2024, Wells Fargo envisions that the gross profit at Procter & Gamble has some probability to remain stable.',
 'On August 21, 2024, Goldman Sachs speculates that the net profit at Microsoft will likely increase in the following fiscal year.',
 'The net profit at Amazon should stay same in 2024/08/21, according to a financial reporter.',
 'According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.',
 'JPMorgan observed that the net profit at Microsoft had risen in September 2023.',
 'According to Google, the gross profit at Facebook would fall in the second quarter of 2026.',
 'Accordin

In [23]:
my_sentences = ["10%", "10.5", "11"]

In [24]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(my_sentences)
print(embeddings.shape)
# [3, 384]

# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)
# tensor([[1.0000, 0.6660, 0.1046],
#         [0.6660, 1.0000, 0.1411],
#         [0.1046, 0.1411, 1.0000]])

KeyboardInterrupt: 